In [1]:
import pystac

from mccn.extent import GeoBoxBuilder
from mccn._types import CubeConfig, FilterConfig, ProcessConfig
from mccn.loader.point import PointLoader, PointLoadConfig
from mccn.parser import parse_item

### Standard Collection

In [2]:
std_collection = pystac.Collection.from_file(
    "tests/files/unit_tests/point/standard/stac/collection.json"
)
std_loader = PointLoader(
    items=[parse_item(item) for item in std_collection.get_items(recursive=True)],
    filter_config=FilterConfig(GeoBoxBuilder.from_collection(std_collection, 120)),
    load_config=PointLoadConfig(interp=None),
)
std_ds = std_loader.load()
std_ds

<xarray.Dataset> Size: 688B
Dimensions:      (time: 2, lat: 6, lon: 6)
Coordinates:
  * time         (time) datetime64[ns] 16B 2025-01-01 2025-01-02
  * lat          (lat) float64 48B -37.91 -37.91 -37.91 -37.9 -37.9 -37.9
  * lon          (lon) float64 48B 144.6 144.6 144.6 144.6 144.6 144.6
Data variables:
    temperature  (time, lat, lon) float64 576B nan nan 22.95 ... 23.5 nan nan

In [3]:
std_ds.sel(lat=-37.90442, lon=144.639644, time="2025-01-01", method="nearest")

<xarray.Dataset> Size: 32B
Dimensions:      ()
Coordinates:
    time         datetime64[ns] 8B 2025-01-01
    lat          float64 8B -37.9
    lon          float64 8B 144.6
Data variables:
    temperature  float64 8B 23.2

### Conversion Alias

In [4]:
conversion_collection = pystac.Collection.from_file(
    "tests/files/unit_tests/point/conversion/stac/collection.json"
)
conversion_loader = PointLoader(
    items=[
        parse_item(item) for item in conversion_collection.get_items(recursive=True)
    ],
    filter_config=FilterConfig(
        GeoBoxBuilder.from_collection(conversion_collection, 120)
    ),
    process_config=ProcessConfig(
        rename_bands={"temperature_F": "temperature"},
        process_bands={"temperature_F": lambda x: (x - 32) * 5 / 9},
    ),
    load_config=PointLoadConfig(interp=None),
)
conversion_ds = conversion_loader.load()
conversion_ds

<xarray.Dataset> Size: 688B
Dimensions:      (time: 2, lat: 6, lon: 6)
Coordinates:
  * time         (time) datetime64[ns] 16B 2025-01-01 2025-01-02
  * lat          (lat) float64 48B -37.91 -37.91 -37.91 -37.9 -37.9 -37.9
  * lon          (lon) float64 48B 144.6 144.6 144.6 144.6 144.6 144.6
Data variables:
    temperature  (time, lat, lon) float64 576B nan nan 22.95 ... 23.5 nan nan

### Use Z

In [5]:
z_loader = PointLoader(
    items=[parse_item(item) for item in std_collection.get_items(recursive=True)],
    filter_config=FilterConfig(GeoBoxBuilder.from_collection(std_collection, 120)),
    cube_config=CubeConfig(use_z=True),
    load_config=PointLoadConfig(interp=None),
)
z_ds = z_loader.load()
z_ds

<xarray.Dataset> Size: 1kB
Dimensions:      (time: 2, lat: 6, lon: 6, alt: 2)
Coordinates:
  * time         (time) datetime64[ns] 16B 2025-01-01 2025-01-02
  * lat          (lat) float64 48B -37.91 -37.91 -37.91 -37.9 -37.9 -37.9
  * lon          (lon) float64 48B 144.6 144.6 144.6 144.6 144.6 144.6
  * alt          (alt) int64 16B 0 10
Data variables:
    temperature  (time, lat, lon, alt) float64 1kB nan nan nan ... nan nan nan

In [6]:
std_ds.sel(lat=-37.90442, lon=144.639644, time="2025-01-01", method="nearest")

<xarray.Dataset> Size: 32B
Dimensions:      ()
Coordinates:
    time         datetime64[ns] 8B 2025-01-01
    lat          float64 8B -37.9
    lon          float64 8B 144.6
Data variables:
    temperature  float64 8B 23.2

### No Date

In [12]:
nodate_collection = pystac.Collection.from_file(
    "tests/files/unit_tests/point/nodate/stac/collection.json"
)

nodate_loader = PointLoader(
    items=[parse_item(item) for item in nodate_collection.get_items(recursive=True)],
    filter_config=FilterConfig(GeoBoxBuilder.from_collection(nodate_collection, 120)),
    cube_config=CubeConfig(use_z=False),
    load_config=PointLoadConfig(interp=None),
    process_config=ProcessConfig(
        rename_bands={"temperature_F": "temperature"},
        process_bands={"temperature_F": lambda x: (x - 32) * 5 / 9},
    ),
)
nodate_ds = nodate_loader.load()
nodate_ds

<xarray.Dataset> Size: 416B
Dimensions:      (time: 1, lat: 12, lon: 3)
Coordinates:
  * time         (time) datetime64[ns] 8B 2024-12-31T14:20:00
  * lat          (lat) float64 96B -37.91 -37.91 -37.91 ... -37.91 -37.91 -37.91
  * lon          (lon) float64 24B 144.6 144.6 144.6
Data variables:
    temperature  (time, lat, lon) float64 288B 22.3 nan nan ... nan nan 23.1